In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-12-10 20:57:50.982382: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 20:57:50.982419: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 20:57:50.982970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 20:57:51.080420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 20:57:52.197856: W tensorflow/compiler/

In [18]:
df = pd.read_excel("dataset/updated_text_data.xlsx")


In [19]:
df_1=df[df['गुनासो वर्ग']=='वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी ']
df_2=df[df['गुनासो वर्ग']=='सोधपुछ, सुझाव, प्रशंसा सम्बन्धी'].sample(2863,replace=True)
df_2 = df_2.reset_index(drop=True)
df_3=df[df['गुनासो वर्ग']=='कर्मचारी सम्वन्धी '].sample(2863,replace=True)
df_3 = df_3.reset_index(drop=True)
df_4=df[df['गुनासो वर्ग']=='स्वास्थ्यसँग सम्बन्धी'].sample(2863,replace=True)
df_4 = df_4.reset_index(drop=True)
df_5=df[df['गुनासो वर्ग']=='अर्थ सबन्धी '].sample(2863,replace=True)
df_5 = df_5.reset_index(drop=True)
df_6=df[df['गुनासो वर्ग']=='खानेपानी सम्बन्धी '].sample(2863,replace=True)
df_6 = df_6.reset_index(drop=True)
df_7=df[df['गुनासो वर्ग']=='सूचना तथा  संचार सम्बन्धी'].sample(2863,replace=True)
df_7 = df_7.reset_index(drop=True)
df_8=df[df['गुनासो वर्ग']=='शान्ति सुरक्षा सम्बन्धी '].sample(2863,replace=True)
df_8 = df_8.reset_index(drop=True)
df_9=df[df['गुनासो वर्ग']=='प्राकृतिक श्रोत/साधन सम्बन्धी'].sample(2863,replace=True)
df_9 =df_9.reset_index(drop=True)
df_10=df[df['गुनासो वर्ग']=='लागु पदार्थ सम्बन्धी '].sample(2863,replace=True)
df_10 = df_10.reset_index(drop=True)
df_11=df[df['गुनासो वर्ग']=='अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी '].sample(2863,replace=True)
df_11 = df_11.reset_index(drop=True)



In [20]:
df_final=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11],axis=0)

In [21]:
# Data Preprocessing
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    text = text.strip('"')

    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    processed_text = ' '.join(filtered_words)
    return processed_text

df_final['गुनासो'] = df_final['गुनासो'].apply(preprocess_text)

In [22]:
# Encoding Labels
le = LabelEncoder()
df_final['label_encoded'] = le.fit_transform(df_final['गुनासो वर्ग'])


In [23]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df_final, test_size=0.2, stratify=df_final['label_encoded'], random_state=42)


In [24]:
# Tokenize the text data
max_words = 10000
max_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['गुनासो'])

In [25]:
train_sequences = tokenizer.texts_to_sequences(train_data['गुनासो'])

test_sequences = tokenizer.texts_to_sequences(test_data['गुनासो'])

In [26]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [27]:
# Load your pre-trained Nepali Word2Vec model
word2vec_model_path = "nepaliW2V_5Million.model"
word2vec_model = Word2Vec.load(word2vec_model_path)

In [28]:
# Create an embedding matrix
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words and word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [29]:
# Build the LSTM model with Word2Vec embeddings
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Bidirectional(LSTM(units=50)))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=len(np.unique(df_final['label_encoded'])), activation='softmax'))

In [30]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20
model.fit(train_padded, train_data['label_encoded'], epochs=epochs, validation_data=(test_padded, test_data['label_encoded']))


Epoch 1/20


788/788 [==============================] - 25s 26ms/step - loss: 1.2821 - accuracy: 0.5701 - val_loss: 0.9680 - val_accuracy: 0.6617
Epoch 2/20
788/788 [==============================] - 18s 23ms/step - loss: 0.9023 - accuracy: 0.6876 - val_loss: 0.8604 - val_accuracy: 0.7030
Epoch 3/20
788/788 [==============================] - 19s 24ms/step - loss: 0.7877 - accuracy: 0.7239 - val_loss: 0.7749 - val_accuracy: 0.7308
Epoch 4/20
788/788 [==============================] - 19s 24ms/step - loss: 0.7199 - accuracy: 0.7463 - val_loss: 0.7653 - val_accuracy: 0.7327
Epoch 5/20
788/788 [==============================] - 19s 25ms/step - loss: 0.6598 - accuracy: 0.7653 - val_loss: 0.7225 - val_accuracy: 0.7528
Epoch 6/20
788/788 [==============================] - 18s 23ms/step - loss: 0.6147 - accuracy: 0.7793 - val_loss: 0.7318 - val_accuracy: 0.7522
Epoch 7/20
788/788 [==============================] - 20s 25ms/step - loss: 0.6019 - accuracy: 0.7854 - val_loss: 0.6879 - val_accuracy: 0.7603
Epo

In [31]:
# Evaluate the model
y_pred_probs = model.predict(test_padded)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Decode the encoded labels back to original classes
y_true = le.inverse_transform(test_data['label_encoded'])
y_pred_decoded = le.inverse_transform(y_pred)

# Print the classification report
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(y_true, y_pred_decoded))


197/197 [==============================] - 3s 9ms/step
Classification Report:
                                           precision    recall  f1-score   support

                             अर्थ सबन्धी        0.89      0.77      0.82       572
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी        0.99      0.97      0.98       573
                       कर्मचारी सम्वन्धी        0.86      0.78      0.82       573
                       खानेपानी सम्बन्धी        0.85      0.72      0.78       573
            प्राकृतिक श्रोत/साधन सम्बन्धी       0.95      0.86      0.90       573
                    लागु पदार्थ सम्बन्धी        0.99      0.97      0.98       572
  वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी        0.32      0.73      0.44       573
                 शान्ति सुरक्षा सम्बन्धी        0.91      0.62      0.74       573
                सूचना तथा  संचार सम्बन्धी       0.92      0.79      0.85       573
          सोधपुछ, सुझाव, प्रशंसा सम्बन्धी       0.76      0.59      0.66       572
        

In [35]:
# New Nepali text
new_text = "यस महानगरपालिकाको सुनाकोठी नखिपोट नख्खुडोल गोकुल आवास कान्तिपुर कोलोनीका वासिन्दाले दिनरात यो धुंवाको सास कहिलेसम्म फेर्नुपर्ने हो हजुर?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 66ms/step
Predicted Class: खानेपानी सम्बन्धी  (Class 3)


In [36]:
# New Nepali text
new_text = "सुनसरी जिल्ला इटहरी उप महानगर भित्र अवस्थित धरान पुग्ने यो बाटो । कुन ठेकेदारले यो काम गर्दा कति पैसा कुम्ल्याउन पाइने हो र आफ्नो निर्वाचन ताका खर्च भएको करोड उठाउन पाइन्छ भनेर त हैन ?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 23ms/step
Predicted Class: कर्मचारी सम्वन्धी  (Class 2)


In [34]:
# New Nepali text
new_text = "इटहरीदेखि पूर्वाञ्चल विश्वविद्यालय, माेरङ हुँदै पूर्व जाने बाटाे जुन लालभित्तीमा गएर टुङ्गिन्छ, पिच गर्नुपर्‍याे। बीचमा दुइटा ठूला खाेला छन्, त्यसमा पुल हाल्नुपर्‍याे। याे बाटाेलाई वैकल्पिक राजमार्गकाे रुपमा विकास गर्नुप्रयो।"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 30ms/step
Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी (Class 9)


In [40]:
# New Nepali text
new_text = "मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग। बाह्रबिसेका मेयरको मनोमानी- दोहोरीमा रमाइलो गरेको बिलसमेत नगरपालिकाबाटै भुक्तानी गर्न दबाब मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग।"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 26ms/step
Predicted Class: अर्थ सबन्धी  (Class 0)
